In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from pyod.models.knn import KNN 

from sklearn.feature_selection import( 
    RFECV,
    SequentialFeatureSelector,
    SelectFromModel
)

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import(
    RandomForestClassifier,
    HistGradientBoostingClassifier
)

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from src.wrapped import Wrapped
from src.train import cross_validate_balancead

In [2]:
wp = Wrapped(
    '../data/row/',
    '../data/processed/',
    '../data/files/'
)

# Importando dados para treinamento

In [3]:
df = wp.load_data('df_instrumentos_features_selecionadas')

In [4]:
# instrumentos de sopro:
inst_sopro = ["clarinet", "trombone", "flute", "trumpet", "saxophone"]
df = df[df['instrumento'].isin(inst_sopro)]

- Definindo as variaveis de treino e o target

In [5]:
le = LabelEncoder()
df['labels'] = le.fit_transform(df.instrumento)

In [6]:
X = df.drop(columns=['labels', 'instrumento','file_name'])
y = df['labels'].to_frame()

# TESTE 1: Balanceamento

- 1° Opção: Balanceamento das classes com SMOTH

In [7]:
t1_models = np.array([
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    LGBMClassifier(),
    MLPClassifier(),
    SVC(),
])

t1_acuracy_models = [cross_validate_balancead(k=5, model=model, X=X, y=y, oversampling=True) for model in t1_models]

dict_results_t1 = {
    'Naive Bayes': t1_acuracy_models[0],
    'KNN': t1_acuracy_models[1],
    'Arvore de Decisão': t1_acuracy_models[2],
    'Floresta Aleatoria': t1_acuracy_models[3],
    'HistGradientBoosting': t1_acuracy_models[4],
    'LIGHTGBM': t1_acuracy_models[5],
    'MLP': t1_acuracy_models[6],
    'SVC': t1_acuracy_models[7],
}

dict_results_t1

Acuracia do modelo GaussianNB() do Fold 0: 0.5279503105590062
Acuracia do modelo GaussianNB() do Fold 1: 0.5714285714285714
Acuracia do modelo GaussianNB() do Fold 2: 0.5652173913043478
Acuracia do modelo GaussianNB() do Fold 3: 0.5434782608695652
Acuracia do modelo GaussianNB() do Fold 4: 0.5981308411214953
Acuracia do modelo KNeighborsClassifier() do Fold 0: 0.35403726708074534
Acuracia do modelo KNeighborsClassifier() do Fold 1: 0.3385093167701863
Acuracia do modelo KNeighborsClassifier() do Fold 2: 0.37577639751552794
Acuracia do modelo KNeighborsClassifier() do Fold 3: 0.3416149068322981
Acuracia do modelo KNeighborsClassifier() do Fold 4: 0.3925233644859813
Acuracia do modelo DecisionTreeClassifier() do Fold 0: 0.43478260869565216
Acuracia do modelo DecisionTreeClassifier() do Fold 1: 0.5279503105590062
Acuracia do modelo DecisionTreeClassifier() do Fold 2: 0.5093167701863354
Acuracia do modelo DecisionTreeClassifier() do Fold 3: 0.4751552795031056
Acuracia do modelo DecisionTree

{'Naive Bayes': 0.5612410750565973,
 'KNN': 0.36049225053694783,
 'Arvore de Decisão': 0.49162167914707533,
 'Floresta Aleatoria': 0.6762214353437433,
 'HistGradientBoosting': 0.6960952767941797,
 'LIGHTGBM': 0.7016834039589017,
 'MLP': 0.43497223350941355,
 'SVC': 0.3238095238095238}

- 2° Opção: Class Weight

In [8]:
models_class_weight = np.array([
    DecisionTreeClassifier(class_weight='balanced'), 
    RandomForestClassifier(class_weight='balanced', random_state = 0, n_jobs = -1), 
    LGBMClassifier(class_weight='balanced'),
])

t1_acuracy_models_class_weight = [cross_validate_balancead(k=5, model=model, X=X, y=y, weight=True) for model in models_class_weight]

dict_results_t1_class_weight = {
    'Arvore de Decisão': t1_acuracy_models_class_weight[0],
    'Floresta Aleatoria': t1_acuracy_models_class_weight[1],
    'LIGHTGBM': t1_acuracy_models_class_weight[2],
}

dict_results_t1_class_weight

Acuracia do modelo DecisionTreeClassifier(class_weight='balanced') do Fold 0: 0.5279503105590062
Acuracia do modelo DecisionTreeClassifier(class_weight='balanced') do Fold 1: 0.5093167701863354
Acuracia do modelo DecisionTreeClassifier(class_weight='balanced') do Fold 2: 0.5186335403726708
Acuracia do modelo DecisionTreeClassifier(class_weight='balanced') do Fold 3: 0.5186335403726708
Acuracia do modelo DecisionTreeClassifier(class_weight='balanced') do Fold 4: 0.48598130841121495
Acuracia do modelo RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=0) do Fold 0: 0.6832298136645962
Acuracia do modelo RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=0) do Fold 1: 0.7018633540372671
Acuracia do modelo RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=0) do Fold 2: 0.7204968944099379
Acuracia do modelo RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=0) do Fold 3: 0.6801242236024845
Acuracia do modelo

{'Arvore de Decisão': 0.5121030939803797,
 'Floresta Aleatoria': 0.7054294615042278,
 'LIGHTGBM': 0.7066736324761518}

# TESTE 2: Remover Outlier

- Treinar modelo de classificação de outilier

In [9]:
detector_outilier = KNN()

new_df = df.drop(columns=['instrumento','file_name'])

detector_outilier.fit(new_df)

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

- Checar numeros de outilers e não outilers<br/>
**0 (False) Não é outiler**<br/>
**1 (True) É outiler**

In [10]:
previsions = detector_outilier.labels_
np.unique(previsions, return_counts=True)

(array([0, 1]), array([1448,  161]))

- Distancia euclidiana dos registros

In [11]:
confiance_previsions = detector_outilier.decision_scores_
confiance_previsions

array([ 573.28682546, 1165.41972437,  529.69517971, ...,  750.32724957,
        748.97427048,  424.90582813])

- Checar os outilers

In [12]:
# pegando indice dos registros que são outilers
outilers_id =[previsions[i] == 1 for i in range(previsions.shape[0])]
outilers = df.iloc[outilers_id,:]
outilers

,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,mfcc0,...,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,file_name,labels
1876,-0.044667,0.260524,0.692127,0.238955,0.029413,0.424056,0.732275,0.106237,0.264656,-178.568253,...,1872.058375,0.862944,2738.513835,-70.011116,-80.000000,0.139929,30932,clarinet,001020_395520.ogg,0
1878,0.089747,0.253008,0.299878,0.225017,0.544357,0.164941,0.079692,0.119684,0.194826,-211.757339,...,1999.856294,1.274141,3282.789275,-58.643265,-71.021973,0.176906,39102,clarinet,001563_195840.ogg,0
1886,0.002779,0.015584,0.311722,0.319367,0.339069,0.368589,0.345589,0.443560,0.344975,-85.485229,...,3154.871456,1.945904,5838.245903,-44.260166,-62.325306,0.232276,51374,clarinet,009170_188160.ogg,0
1888,0.108679,0.066236,0.484411,0.352953,0.433525,0.316416,0.265250,0.316138,0.292531,-121.484428,...,1992.119222,1.766695,4726.313261,-42.178364,-41.680836,0.041709,9216,clarinet,009884_138240.ogg,0
1897,0.040330,0.013987,0.226904,0.302980,0.674932,0.560282,0.423103,0.323710,0.402272,-124.647331,...,1949.869060,1.131823,3117.917883,-63.385639,-78.626289,0.151718,33537,clarinet,015674_115200.ogg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7151,-0.215709,0.074658,0.095596,0.119745,0.361354,0.896803,0.408373,0.369947,0.177898,-387.084808,...,3394.058894,1.074963,6651.736344,-53.667458,-76.665230,0.194558,43089,trumpet,120220_76800.ogg,4
7153,0.095131,0.165130,0.409808,0.178644,0.131329,0.086917,0.195207,0.238547,0.105908,-196.057907,...,1466.013189,1.140587,2337.326779,-75.481300,-79.999992,0.101459,22454,trumpet,125119_11520.ogg,4
7156,-0.148693,0.003055,0.543909,0.820975,0.261659,0.262984,0.201507,0.296120,0.160154,-182.133682,...,2612.295812,0.958099,4009.572352,-64.094101,-80.000000,0.170468,37702,trumpet,130811_399360.ogg,4
7159,-0.040215,0.031151,0.587902,0.642199,0.661011,0.675489,0.697853,0.718583,0.482416,-276.234711,...,285.508118,0.854189,403.660127,-16.440603,-14.334520,0.014532,3213,trumpet,138998_691200.ogg,4


In [13]:
outilers.instrumento.value_counts()

flute        103
clarinet      20
saxophone     19
trumpet       10
trombone       9
Name: instrumento, dtype: int64

- Agora vamos remover os outilers da nossa base

In [14]:
# pegando indice dos registros que não são outilers
outilers_id =[previsions[i] == 0 for i in range(previsions.shape[0])]
df_train = df.iloc[outilers_id,:]
df_train

,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,mfcc0,...,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,instrumento,file_name,labels
1875,0.060383,0.017910,0.319152,0.367783,0.465361,0.399524,0.357706,0.343498,0.282757,8.203455,...,2328.546611,1.093156,5092.952266,-52.844872,-68.585281,0.105613,23362,clarinet,000726_72960.ogg,0
1877,-0.102407,-0.053821,0.007352,0.016456,0.251515,0.982690,0.649991,0.123668,0.015163,-170.240860,...,1571.665681,0.959548,3635.089465,-61.261280,-78.551147,0.036253,8015,clarinet,001210_184320.ogg,0
1879,0.137429,-0.165245,0.004996,0.004829,0.009536,0.313501,1.000000,0.247865,0.029246,-200.479965,...,1302.617113,0.927032,2510.017073,-61.047974,-77.810928,0.029619,6548,clarinet,001630_134400.ogg,0
1880,0.236781,-0.104811,0.360671,0.143619,0.062925,0.141207,0.458111,0.159797,0.037194,-225.307404,...,1104.750913,1.166350,1975.259208,-66.297119,-78.115021,0.066677,14813,clarinet,003872_0.ogg,0
1881,0.251144,0.276060,0.757470,0.276578,0.013081,0.137285,0.914434,0.177750,0.035843,-196.740616,...,930.040460,0.972946,1324.891525,-62.295158,-79.964806,0.063943,14130,clarinet,005201_99840.ogg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7161,0.031326,0.036042,0.281165,0.467478,0.383710,0.385898,0.358991,0.379168,0.286116,-221.750122,...,896.653459,1.242603,1782.859289,-42.632786,-42.830780,0.041767,9231,trumpet,141573_157440.ogg,4
7162,0.024069,-0.064162,0.038717,0.051984,0.156484,0.528343,0.865360,0.228249,0.063890,-213.012329,...,1280.731144,1.535133,1930.644010,-52.891537,-79.643761,0.078466,17360,trumpet,149060_46080.ogg,4
7163,0.046916,0.068088,0.517141,0.282860,0.547268,0.444383,0.516097,0.406917,0.322084,-35.486549,...,2309.708871,1.049440,5199.019529,-37.911655,-57.435429,0.095421,21114,trumpet,149506_92160.ogg,4
7164,0.109233,0.002179,0.271734,0.290186,0.454663,0.332584,0.266866,0.341721,0.481044,-134.787796,...,2370.998419,1.259253,5153.205266,-53.025776,-79.120621,0.126226,27933,trumpet,150659_195840.ogg,4


- Treinar modelos sem outlier e balanceameto

In [15]:
X_rm_outilers = df_train.drop(columns=['labels', 'instrumento','file_name'])
y_rm_outilers = df_train['labels'].to_frame()

models = np.array([
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    LGBMClassifier(),,
    MLPClassifier(),
    SVC(),
])

t2_acuracy_models = [cross_validate_balancead(k=5, model=model, X=X_rm_outilers,  y=y_rm_outilers) for model in models]

dict_results_t2 = {
    'Naive Bayes': t2_acuracy_models[0],
    'KNN': t2_acuracy_models[1],
    'Arvore de Decisão': t2_acuracy_models[2],
    'Floresta Aleatoria': t2_acuracy_models[3],
    'HistGradientBoosting': t2_acuracy_models[4],
    'LIGHTGBM': t2_acuracy_models[5],
    'MLP': t2_acuracy_models[6],
    'SVC': t2_acuracy_models[7],
}

dict_results_t2

Acuracia do modelo GaussianNB() do Fold 0: 0.5379310344827586
Acuracia do modelo GaussianNB() do Fold 1: 0.6448275862068965
Acuracia do modelo GaussianNB() do Fold 2: 0.596551724137931
Acuracia do modelo GaussianNB() do Fold 3: 0.5847750865051903
Acuracia do modelo GaussianNB() do Fold 4: 0.6332179930795848
Acuracia do modelo KNeighborsClassifier() do Fold 0: 0.4517241379310345
Acuracia do modelo KNeighborsClassifier() do Fold 1: 0.4862068965517241
Acuracia do modelo KNeighborsClassifier() do Fold 2: 0.5
Acuracia do modelo KNeighborsClassifier() do Fold 3: 0.47750865051903113
Acuracia do modelo KNeighborsClassifier() do Fold 4: 0.5363321799307958
Acuracia do modelo DecisionTreeClassifier() do Fold 0: 0.5310344827586206
Acuracia do modelo DecisionTreeClassifier() do Fold 1: 0.5413793103448276
Acuracia do modelo DecisionTreeClassifier() do Fold 2: 0.5724137931034483
Acuracia do modelo DecisionTreeClassifier() do Fold 3: 0.5501730103806228
Acuracia do modelo DecisionTreeClassifier() do Fo

{'Naive Bayes': 0.5994606848824723,
 'KNN': 0.4903543729865172,
 'Arvore de Decisão': 0.5552630950960505,
 'Floresta Aleatoria': 0.7134017420355567,
 'HistGradientBoosting': 0.7023553275265482,
 'LIGHTGBM': 0.7161651354253669,
 'MLP': 0.4765564968380862,
 'SVC': 0.5145090084715427}

# TESTE 3: Features de Maior Correlação

In [16]:
df.corr()

,tonnetz0,tonnetz1,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,mfcc0,...,spectral_band3,spectral_band4,spectral_centroid,spectral_onset,spectral_rolloff,spectogram,mel_spectogram,zero_crossing_rate,zero_crossing,labels
tonnetz0,1.000000,0.099321,-0.063724,-0.161056,0.216762,-0.186788,0.185363,-0.142416,-0.078336,-0.004246,...,-0.062137,-0.059795,-0.076439,-0.088682,-0.060949,-0.011419,-0.032382,-0.072001,-0.072011,0.006135
tonnetz1,0.099321,1.000000,0.251187,-0.170835,0.112380,-0.114633,-0.189572,0.044960,-0.259020,-0.037514,...,-0.050582,-0.043721,-0.057252,-0.044823,-0.058028,0.016126,-0.001446,-0.051143,-0.051138,-0.011587
chroma1,-0.063724,0.251187,1.000000,0.453877,0.050341,0.024621,0.053949,0.067711,0.020134,0.140978,...,0.045435,0.035970,0.045174,0.060065,0.062918,0.167669,0.168858,0.048772,0.048750,0.113477
chroma2,-0.161056,-0.170835,0.453877,1.000000,0.445546,0.232762,0.107829,0.132620,0.185293,0.188667,...,0.112751,0.094106,0.114009,0.083581,0.140751,0.141301,0.172793,0.078678,0.078688,0.126276
chroma3,0.216762,0.112380,0.050341,0.445546,1.000000,0.468904,0.119251,0.091268,0.086475,0.157098,...,0.098766,0.085877,0.076699,0.059752,0.108206,0.123763,0.154553,0.031537,0.031534,0.115415
chroma4,-0.186788,-0.114633,0.024621,0.232762,0.468904,1.000000,0.496796,0.237628,0.133118,0.154068,...,0.086485,0.072389,0.075539,0.114269,0.102512,0.167613,0.200833,0.041606,0.041638,0.147380
chroma5,0.185363,-0.189572,0.053949,0.107829,0.119251,0.496796,1.000000,0.502817,0.183457,0.183314,...,0.048987,0.042961,0.012802,0.110436,0.054665,0.204967,0.233346,-0.010095,-0.010080,0.214129
chroma6,-0.142416,0.044960,0.067711,0.132620,0.091268,0.237628,0.502817,1.000000,0.476589,0.170841,...,0.070747,0.069245,0.017939,0.120099,0.055047,0.214744,0.245109,-0.013190,-0.013133,0.182057
chroma7,-0.078336,-0.259020,0.020134,0.185293,0.086475,0.133118,0.183457,0.476589,1.000000,0.180398,...,0.121547,0.109887,0.073903,0.099535,0.120975,0.130105,0.198958,0.034481,0.034543,0.121271
mfcc0,-0.004246,-0.037514,0.140978,0.188667,0.157098,0.154068,0.183314,0.170841,0.180398,1.000000,...,0.482809,0.439897,0.429049,0.336226,0.506348,0.301315,0.245305,0.226500,0.226395,0.084968


In [17]:
df_corr = df.corr().unstack().reset_index().dropna()
df_corr.rename(columns = {'level_0': 'features_a', 'level_1': 'features_b', 0:'correlacao'}, inplace = True)
df_corr

,features_a,features_b,correlacao
0,tonnetz0,tonnetz0,1.000000
1,tonnetz0,tonnetz1,0.099321
2,tonnetz0,chroma1,-0.063724
3,tonnetz0,chroma2,-0.161056
4,tonnetz0,chroma3,0.216762
...,...,...,...
2020,labels,spectogram,0.102177
2021,labels,mel_spectogram,0.092350
2022,labels,zero_crossing_rate,-0.161517
2023,labels,zero_crossing,-0.161526


- Correlação Positia

In [18]:
corr_posit = df_corr.query('correlacao > 0.7 and correlacao < 1.0')
corr_posit

,features_a,features_b,correlacao
1565,spectral_band2,spectral_band3,0.977219
1566,spectral_band2,spectral_band4,0.922914
1567,spectral_band2,spectral_centroid,0.822786
1569,spectral_band2,spectral_rolloff,0.917366
1609,spectral_band3,spectral_band2,0.977219
1611,spectral_band3,spectral_band4,0.983204
1612,spectral_band3,spectral_centroid,0.722382
1614,spectral_band3,spectral_rolloff,0.821823
1654,spectral_band4,spectral_band2,0.922914
1655,spectral_band4,spectral_band3,0.983204


In [19]:
corr_posit.features_a.unique()

array(['spectral_band2', 'spectral_band3', 'spectral_band4',
       'spectral_centroid', 'spectral_rolloff', 'spectogram',
       'mel_spectogram', 'zero_crossing_rate', 'zero_crossing'],
      dtype=object)

In [20]:
X =  df[corr_posit.features_a.unique()]
y = df['labels'].to_frame()

models = np.array([
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    LGBMClassifier(),
    # XGBClassifier(),
    MLPClassifier(),
    SVC(),
])

t3_acuracy_models_corr_posit = [cross_validate_balancead(k=5, model=model, X=X, y=y) for model in models]

dict_results_t3_corr_posit = {
    'Naive Bayes': t3_acuracy_models_corr_posit[0],
    'KNN': t3_acuracy_models_corr_posit[1],
    'Arvore de Decisão': t3_acuracy_models_corr_posit[2],
    'Floresta Aleatoria': t3_acuracy_models_corr_posit[3],
    'HistGradientBoosting': t3_acuracy_models_corr_posit[4],
    'LIGHTGBM': t3_acuracy_models_corr_posit[5],
    # 'XGB': t3_acuracy_models_corr_posit[6],
    'MLP': t3_acuracy_models_corr_posit[6],
    'SVC': t3_acuracy_models_corr_posit[7],
}

dict_results_t3_corr_posit

Acuracia do modelo GaussianNB() do Fold 0: 0.4720496894409938
Acuracia do modelo GaussianNB() do Fold 1: 0.5
Acuracia do modelo GaussianNB() do Fold 2: 0.5217391304347826
Acuracia do modelo GaussianNB() do Fold 3: 0.4906832298136646
Acuracia do modelo GaussianNB() do Fold 4: 0.5171339563862928
Acuracia do modelo KNeighborsClassifier() do Fold 0: 0.46273291925465837
Acuracia do modelo KNeighborsClassifier() do Fold 1: 0.4751552795031056
Acuracia do modelo KNeighborsClassifier() do Fold 2: 0.5062111801242236
Acuracia do modelo KNeighborsClassifier() do Fold 3: 0.484472049689441
Acuracia do modelo KNeighborsClassifier() do Fold 4: 0.5420560747663551
Acuracia do modelo DecisionTreeClassifier() do Fold 0: 0.45652173913043476
Acuracia do modelo DecisionTreeClassifier() do Fold 1: 0.43788819875776397
Acuracia do modelo DecisionTreeClassifier() do Fold 2: 0.468944099378882
Acuracia do modelo DecisionTreeClassifier() do Fold 3: 0.4813664596273292
Acuracia do modelo DecisionTreeClassifier() do F

{'Naive Bayes': 0.5003212012151468,
 'KNN': 0.4941255006675568,
 'Arvore de Decisão': 0.4630250962636172,
 'Floresta Aleatoria': 0.5929393781080087,
 'HistGradientBoosting': 0.5867204581954683,
 'LIGHTGBM': 0.570561715137091,
 'MLP': 0.4002341285965829,
 'SVC': 0.511518739962462}

- Correlação Negativa

In [21]:
corr_negat = df_corr.query('correlacao < 0 and correlacao < -0.66')
corr_negat

,features_a,features_b,correlacao
484,mfcc1,spectral_band2,-0.798403
485,mfcc1,spectral_band3,-0.745708
486,mfcc1,spectral_band4,-0.666942
487,mfcc1,spectral_centroid,-0.861014
489,mfcc1,spectral_rolloff,-0.831580
1540,spectral_band2,mfcc1,-0.798403
1585,spectral_band3,mfcc1,-0.745708
1630,spectral_band4,mfcc1,-0.666942
1675,spectral_centroid,mfcc1,-0.861014
1765,spectral_rolloff,mfcc1,-0.831580


In [22]:
X =  df[corr_negat.features_b.unique()]
y = df['labels'].to_frame()

models = np.array([
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    LGBMClassifier(),
    XGBClassifier(),
    MLPClassifier(),
    SVC(),
])

t3_acuracy_models_corr_neg = [cross_validate_balancead(k=5, model=model, X=X, y=y) for model in models]

dict_results_t3_corr_neg = {
    'Naive Bayes': t3_acuracy_models_corr_neg[0],
    'KNN': t3_acuracy_models_corr_neg[1],
    'Arvore de Decisão': t3_acuracy_models_corr_neg[2],
    'Floresta Aleatoria': t3_acuracy_models_corr_neg[3],
    'HistGradientBoosting': t3_acuracy_models_corr_neg[4],
    'LIGHTGBM': t3_acuracy_models_corr_neg[5],
    # 'XGB': t3_acuracy_models_corr_neg[6],;
    'MLP': t3_acuracy_models_corr_neg[6],
    'SVC': t3_acuracy_models_corr_neg[7],
}

dict_results_t3_corr_neg

Acuracia do modelo GaussianNB() do Fold 0: 0.43478260869565216
Acuracia do modelo GaussianNB() do Fold 1: 0.4658385093167702
Acuracia do modelo GaussianNB() do Fold 2: 0.48757763975155277
Acuracia do modelo GaussianNB() do Fold 3: 0.4720496894409938
Acuracia do modelo GaussianNB() do Fold 4: 0.4953271028037383
Acuracia do modelo KNeighborsClassifier() do Fold 0: 0.43478260869565216
Acuracia do modelo KNeighborsClassifier() do Fold 1: 0.4720496894409938
Acuracia do modelo KNeighborsClassifier() do Fold 2: 0.5496894409937888
Acuracia do modelo KNeighborsClassifier() do Fold 3: 0.4813664596273292
Acuracia do modelo KNeighborsClassifier() do Fold 4: 0.5109034267912772
Acuracia do modelo DecisionTreeClassifier() do Fold 0: 0.4658385093167702
Acuracia do modelo DecisionTreeClassifier() do Fold 1: 0.4658385093167702
Acuracia do modelo DecisionTreeClassifier() do Fold 2: 0.5372670807453416
Acuracia do modelo DecisionTreeClassifier() do Fold 3: 0.484472049689441
Acuracia do modelo DecisionTreeC

{'Naive Bayes': 0.4711151100017415,
 'KNN': 0.4897583251098082,
 'Arvore de Decisão': 0.4729262204678702,
 'Floresta Aleatoria': 0.5718039511619357,
 'HistGradientBoosting': 0.5606122172558581,
 'LIGHTGBM': 0.5599949691375941,
 'MLP': 0.5637255471062866,
 'SVC': 0.42200615313171186}

- Se combinarmos as duas abordagens ?

In [23]:
features = np.append(corr_posit.features_b.values, corr_negat.features_b.values)
features = np.unique(features)

X = df[features]
y = df['labels'].to_frame()

models = np.array([
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    LGBMClassifier(),
    XGBClassifier(),
    MLPClassifier(),
    SVC(),
])

t3_combine_corr = [cross_validate_balancead(k=5, model=model, X=X, y=y) for model in models]

dict_results_t3_cb = {
    'Naive Bayes': t3_combine_corr[0],
    'KNN': t3_combine_corr[1],
    'Arvore de Decisão': t3_combine_corr[2],
    'Floresta Aleatoria': t3_combine_corr[3],
    'HistGradientBoosting': t3_combine_corr[4],
    'LIGHTGBM': t3_combine_corr[5],
    'XGB': t3_combine_corr[6],
    'MLP': t3_combine_corr[7],
    'SVC': t3_combine_corr[8],
}

dict_results_t3_cb

Acuracia do modelo GaussianNB() do Fold 0: 0.48757763975155277
Acuracia do modelo GaussianNB() do Fold 1: 0.5062111801242236
Acuracia do modelo GaussianNB() do Fold 2: 0.5124223602484472
Acuracia do modelo GaussianNB() do Fold 3: 0.4813664596273292
Acuracia do modelo GaussianNB() do Fold 4: 0.5233644859813084
Acuracia do modelo KNeighborsClassifier() do Fold 0: 0.4658385093167702
Acuracia do modelo KNeighborsClassifier() do Fold 1: 0.4782608695652174
Acuracia do modelo KNeighborsClassifier() do Fold 2: 0.5062111801242236
Acuracia do modelo KNeighborsClassifier() do Fold 3: 0.484472049689441
Acuracia do modelo KNeighborsClassifier() do Fold 4: 0.5420560747663551
Acuracia do modelo DecisionTreeClassifier() do Fold 0: 0.43478260869565216
Acuracia do modelo DecisionTreeClassifier() do Fold 1: 0.4658385093167702
Acuracia do modelo DecisionTreeClassifier() do Fold 2: 0.5279503105590062
Acuracia do modelo DecisionTreeClassifier() do Fold 3: 0.4937888198757764
Acuracia do modelo DecisionTreeCl

{'Naive Bayes': 0.5021884251465722,
 'KNN': 0.4953677366924015,
 'Arvore de Decisão': 0.4785530465741762,
 'Floresta Aleatoria': 0.6233838354521003,
 'HistGradientBoosting': 0.6010139122694994,
 'LIGHTGBM': 0.6171629805924808,
 'XGB': 0.6121862967047852,
 'MLP': 0.43570751339950853,
 'SVC': 0.5096515160310365}

# TESTE4: Recursive Feature Elimination - (RFE)

- RFE

In [24]:
X = df.drop(columns=['instrumento','file_name','labels'])
y = df['labels'].to_frame()

clf = DecisionTreeClassifier(max_depth=17,
                             max_features='auto', 
                             min_samples_leaf=2,
                             min_samples_split=3)
                               
                               
estimators = RFECV(estimator=clf, scoring='accuracy', cv=5)

# fit rfecv
rfecv_data = estimators.fit(X, y)

# gorresponde à posição do ranking do i-ésimo recurso
ranking_features = rfecv_data.ranking_

# A máscara dos recursos selecionados.
support_features = rfecv_data.support_

# filtro
features_selects = X.columns[support_features]

In [25]:
features_selects

Index(['chroma2', 'chroma6', 'mfcc0', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4',
       'mfcc7', 'mfcc8', 'mfcc10', 'mfcc16', 'mfcc18', 'spectral_band2',
       'spectral_onset', 'zero_crossing_rate'],
      dtype='object')

In [26]:
ranking_features.shape

(44,)

- Count Ranking Features

In [27]:
# Couts da features import 
unique, frequency = np.unique(ranking_features, return_counts=True)
summary = np.asarray((unique, frequency)).T
summary

array([[ 1, 15],
       [ 2,  1],
       [ 3,  1],
       [ 4,  1],
       [ 5,  1],
       [ 6,  1],
       [ 7,  1],
       [ 8,  1],
       [ 9,  1],
       [10,  1],
       [11,  1],
       [12,  1],
       [13,  1],
       [14,  1],
       [15,  1],
       [16,  1],
       [17,  1],
       [18,  1],
       [19,  1],
       [20,  1],
       [21,  1],
       [22,  1],
       [23,  1],
       [24,  1],
       [25,  1],
       [26,  1],
       [27,  1],
       [28,  1],
       [29,  1],
       [30,  1]])

- Treinamento de Modelo

In [28]:
models = np.array([
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    LGBMClassifier(),
    XGBClassifier(),
    MLPClassifier(),
    SVC(),
])

t4_acuracy_models = [cross_validate_balancead(k=5, 
                                              model=model,
                                              X=X, 
                                              y=y, 
                                              oversampling=True) for model in models]

dict_results_t4 = {
    'Naive Bayes': t4_acuracy_models[0],
    'KNN': t4_acuracy_models[1],
    'Arvore de Decisão': t4_acuracy_models[2],
    'Floresta Aleatoria': t4_acuracy_models[3],
    'HistGradientBoosting': t4_acuracy_models[4],
    'LIGHTGBM': t4_acuracy_models[5],
    'XGB': t4_acuracy_models[6],
    'MLP': t4_acuracy_models[7],
    'SVC': t4_acuracy_models[8],
}

dict_results_t4

Acuracia do modelo GaussianNB() do Fold 0: 0.5279503105590062
Acuracia do modelo GaussianNB() do Fold 1: 0.5714285714285714
Acuracia do modelo GaussianNB() do Fold 2: 0.5652173913043478
Acuracia do modelo GaussianNB() do Fold 3: 0.5434782608695652
Acuracia do modelo GaussianNB() do Fold 4: 0.5981308411214953
Acuracia do modelo KNeighborsClassifier() do Fold 0: 0.35403726708074534
Acuracia do modelo KNeighborsClassifier() do Fold 1: 0.3385093167701863
Acuracia do modelo KNeighborsClassifier() do Fold 2: 0.37577639751552794
Acuracia do modelo KNeighborsClassifier() do Fold 3: 0.3416149068322981
Acuracia do modelo KNeighborsClassifier() do Fold 4: 0.3925233644859813
Acuracia do modelo DecisionTreeClassifier() do Fold 0: 0.453416149068323
Acuracia do modelo DecisionTreeClassifier() do Fold 1: 0.515527950310559
Acuracia do modelo DecisionTreeClassifier() do Fold 2: 0.515527950310559
Acuracia do modelo DecisionTreeClassifier() do Fold 3: 0.45962732919254656
Acuracia do modelo DecisionTreeCla

{'Naive Bayes': 0.5612410750565973,
 'KNN': 0.36049225053694783,
 'Arvore de Decisão': 0.4916236140941545,
 'Floresta Aleatoria': 0.669385267312939,
 'HistGradientBoosting': 0.6960952767941797,
 'LIGHTGBM': 0.7016834039589017,
 'XGB': 0.6787155821288288,
 'MLP': 0.4380101004237534,
 'SVC': 0.3238095238095238}

# Teste 5: SequentialFeatureSelector

### Forward

In [29]:
def selectorSequential(k, model_estimator, n_features, X, y):
    sfs = SequentialFeatureSelector(
        cv=k, 
        # direction = 'forward',
        n_features_to_select = n_features,
        estimator=model_estimator
    ).fit(X, y)
    
    mask_feature = sfs.get_support()
    
    return X[X.columns[mask_feature]]

In [43]:
_X = selectorSequential(5, DecisionTreeClassifier(), 20, X, y)
_X 

,tonnetz0,chroma1,chroma5,mfcc1,mfcc3,mfcc7,mfcc8,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc18,mfcc19,chroma9,spectral_centroid,spectral_rolloff,mel_spectogram,zero_crossing_rate,zero_crossing
1875,0.060383,0.319152,0.357706,97.471786,17.726433,6.797719,-12.122009,-5.698319,-4.186620,-9.178421,-1.513219,-7.577986,-6.443947,-0.662083,0.303745,2328.546611,5092.952266,-68.585281,0.105613,23362
1876,-0.044667,0.692127,0.732275,61.911442,5.291811,-8.966903,-17.932789,-0.170921,-4.303693,-22.350485,-7.793060,-8.507512,-23.979158,-16.185549,0.131283,1872.058375,2738.513835,-80.000000,0.139929,30932
1877,-0.102407,0.007352,0.649991,107.638947,19.598055,6.002571,0.851709,-19.618681,-17.366951,-20.012222,-7.600814,-7.568542,-7.859767,-5.318481,0.012319,1571.665681,3635.089465,-78.551147,0.036253,8015
1878,0.089747,0.299878,0.079692,120.258324,33.841656,8.346146,-18.452084,-17.178339,13.240421,7.321417,-0.485359,-17.735708,11.148084,-0.458791,0.157988,1999.856294,3282.789275,-71.021973,0.176906,39102
1879,0.137429,0.004996,1.000000,123.996521,17.919901,2.746356,-12.056657,-8.135055,-11.603746,-14.456120,-9.266733,-11.163033,-0.551916,6.183865,0.006446,1302.617113,2510.017073,-77.810928,0.029619,6548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7161,0.031326,0.281165,0.358991,206.371201,31.268805,4.466880,5.603338,-0.550061,-0.846498,3.068826,0.369699,3.024948,-2.675308,-1.034018,0.503876,896.653459,1782.859289,-42.830780,0.041767,9231
7162,0.024069,0.038717,0.865360,157.358063,-22.252892,-6.040964,-9.734241,1.526627,22.664223,11.602703,3.469329,-10.473911,-3.939756,-15.273481,0.073563,1280.731144,1930.644010,-79.643761,0.078466,17360
7163,0.046916,0.517141,0.516097,87.706017,19.628988,13.540272,-1.963343,-4.630041,-0.351327,-0.043541,5.902341,-8.385210,1.184325,5.494852,0.560932,2309.708871,5199.019529,-57.435429,0.095421,21114
7164,0.109233,0.271734,0.266866,85.668198,27.054415,7.659579,-4.940868,-9.414302,-6.928515,-8.427592,0.066050,-0.314506,-5.837808,-1.235146,0.468994,2370.998419,5153.205266,-79.120621,0.126226,27933


In [44]:
models = np.array([
    GaussianNB(),
    KNeighborsClassifier(), 
    DecisionTreeClassifier(), 
    RandomForestClassifier(), 
    HistGradientBoostingClassifier(),
    LGBMClassifier(),
    XGBClassifier(),
    MLPClassifier(),
    SVC(),
])

t5_acuracy_models = [cross_validate_balancead(k=5, model=model, X=_X, y=y) for model in models]

dict_results_t5 = {
    'Naive Bayes':t5_acuracy_models[0],
    'KNN':t5_acuracy_models[1],
    'Arvore de Decisão': t5_acuracy_models[2],
    'Floresta Aleatoria':t5_acuracy_models[3],
    'HistGradientBoosting':t5_acuracy_models[4],
    'LIGHTGBM': t5_acuracy_models[5],
    'XGB': t5_acuracy_models[6],
    'MLP': t5_acuracy_models[7],
    'SVC': t5_acuracy_models[8],
}

dict_results_t5

Acuracia do modelo GaussianNB() do Fold 0: 0.5745341614906833
Acuracia do modelo GaussianNB() do Fold 1: 0.6273291925465838
Acuracia do modelo GaussianNB() do Fold 2: 0.6024844720496895
Acuracia do modelo GaussianNB() do Fold 3: 0.5838509316770186
Acuracia do modelo GaussianNB() do Fold 4: 0.632398753894081
Acuracia do modelo KNeighborsClassifier() do Fold 0: 0.46273291925465837
Acuracia do modelo KNeighborsClassifier() do Fold 1: 0.4472049689440994
Acuracia do modelo KNeighborsClassifier() do Fold 2: 0.5
Acuracia do modelo KNeighborsClassifier() do Fold 3: 0.4751552795031056
Acuracia do modelo KNeighborsClassifier() do Fold 4: 0.4984423676012461
Acuracia do modelo DecisionTreeClassifier() do Fold 0: 0.562111801242236
Acuracia do modelo DecisionTreeClassifier() do Fold 1: 0.5807453416149069
Acuracia do modelo DecisionTreeClassifier() do Fold 2: 0.577639751552795
Acuracia do modelo DecisionTreeClassifier() do Fold 3: 0.6304347826086957
Acuracia do modelo DecisionTreeClassifier() do Fold

{'Naive Bayes': 0.6041195023316112,
 'KNN': 0.4767071070606219,
 'Arvore de Decisão': 0.5842050269925118,
 'Floresta Aleatoria': 0.7073005553298117,
 'HistGradientBoosting': 0.7104061453919235,
 'LIGHTGBM': 0.7085427913546565,
 'XGB': 0.697352992395658,
 'MLP': 0.48792786517288744,
 'SVC': 0.49783866411253647}